In [ ]:
'''
This script runs on Google Colab and performs the following tasks:
1. Load the previously prebuilt Chroma vector database from Google Drive.
2. Tokenize the Vietnamese query using PyVi's ViTokenizer because Vietnamese has special syntax rules,
   and the bkai bi-encoder requires word-segmented input before embedding.
3. Compute the embedding for the query and retrieve the top-5 most similar documents based on cosine similarity.
4. Display the retrieved documents along with their similarity scores.
5. Comment and recommendation
'''


In [ ]:
# Install required libraries
!pip install -qU langchain-huggingface
!pip install -qU "langchain-chroma>=0.1.2"
!pip install pyvi

In [ ]:
# Mount Google Drive to access stored vector database
from google.colab import drive
drive.mount('/content/drive')

# Define source and destination paths for Chroma vector database
source_folder = "/content/drive/My Drive/vectordb"
destination_folder = "/content/vectordb"

# Copy the vector database from Google Drive to local storage
!rsync -av "/content/drive/My Drive/vectordb" /content/


In [ ]:
# Import necessary modules
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_chroma import Chroma
from pyvi import ViTokenizer
from langchain_huggingface import HuggingFaceEmbeddings

# Load the pre-trained embedding model
embeddings = HuggingFaceEmbeddings(model_name='bkai-foundation-models/vietnamese-bi-encoder')

# Load the existing Chroma vector database
vector_db = Chroma(persist_directory="/content/vectordb", embedding_function=embeddings)


In [ ]:
# This is where you can test with your own input
raw_query = ""

# Tokenize the query for proper processing
tokenized_query = ViTokenizer.tokenize(raw_query)

if isinstance(tokenized_query, list):
    tokenized_query = " ".join(tokenized_query) # Just to ensure the query is a string before embedding
query_str = tokenized_query

# Compute the embedding for the tokenized query
query_embedding = embeddings.embed_query(query_str)

# Retrieve the top-5 most relevant documents based on vector similarity
results = vector_db.similarity_search_by_vector(query_embedding, k=5)

# Extract embeddings of retrieved documents for similarity calculation
retrieved_embeddings = [embeddings.embed_query(doc.page_content) for doc in results]
retrieved_embeddings = np.array(retrieved_embeddings)
query_embedding_array = np.array(query_embedding).reshape(1, -1)

# Compute cosine similarity between query and retrieved document embeddings
similarities = cosine_similarity(query_embedding_array, retrieved_embeddings)[0]

# Display retrieved results with similarity scores
for i, (doc, score) in enumerate(zip(results, similarities)):
    print(f"Rank {i+1}:")
    print(f"File Path: {doc.metadata['file_path']}")
    print(f"Cosine Similarity: {score:.4f}")
    print(f"Content Preview: {doc.page_content}...")
    print("-" * 80)

My comment and suggestion will be based on the above output. You can see 2 examples below.

From the query results, several observations can be made regarding the model’s performance and its limitations:

1. Chunking Strategy Issues

The chosen chunk size (2000 tokens) is relatively large, while the overlap (20 tokens) is quite small. This might lead to missing relevant contextual information when performing similarity searches. Larger chunks can make it difficult for the model to pinpoint specific answers, while a small overlap reduces the continuity between adjacent text segments, potentially causing lower cosine similarity scores. A more balanced chunking strategy—perhaps smaller chunk sizes with a slightly larger overlap—could improve retrieval effectiveness.

2. Cosine Similarity and Ranking Performance

The cosine similarity scores are relatively low (see examples below), which suggests that the retrieved documents do not perfectly match the query’s intent. This may be due to the model's embedding quality, the chunking method, or both. Despite containing the correct information, the results lack strong semantic alignment with the query, which could lead to less reliable retrieval performance.

3. Embedding Model Limitations

The 'bkai-foundation-models/vietnamese-bi-encoder' used for embedding may not be the most optimal choice for this task. A better-performing model could be one that is specifically fine-tuned for Vietnamese legal texts or question-answering tasks.

Potential Improvements

a. Experimenting with different embedding models: Testing other pre-trained embeddings that better capture semantic similarities, I personally recommend dangvantuan/vietnamese-embedding

b. Adjusting chunking strategies: Reducing chunk size while increasing overlap to ensure key information is well-represented in embeddings.

c. (Optional, may out of scope) I also suggest applying other techniques, such as routing database, for better perfomance



In [ ]:
'''
Query 1: "Bộ Quốc phòng có trách nhiệm tổ chức thực hiện công tác bảo vệ bí mật nhà nước trong các cơ quan, đơn vị thuộc phạm vi nào?"
Rank 1:
File Path: /content/vbpl/full_5585.html
Cosine Similarity: 0.6638
Content Preview: , xử_lý vi_phạm và giải_quyết khiếu_nại , tố_cáo trong lĩnh_vực bảo_vệ bí_mật nhà_nước ; 6 . Sơ_kết , tổng_kết công_tác bảo_vệ bí_mật nhà_nước . Điều 111 . Chính_phủ thống_nhất quản_lý nhà_nước về bảo_vệ bí_mật nhà_nước . 2 . Bộ Công_an chịu trách_nhiệm trước Chính_phủ thực_hiện quản_lý nhà_nước về bảo_vệ bí_mật nhà_nước và có nhiệm_vụ , quyền_hạn sau đây : a . Trình Chính_phủ , Thủ_tướng Chính_phủ dự_thảo các văn_bản quy_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước . b . Hướng_dẫn cơ_quan , tổ_chức lập danh_mục bí_mật nhà_nước và thực_hiện công_tác bảo_vệ bí_mật nhà_nước ; c . Thẩm_định việc lập và giải mật_danh_mục bí_mật nhà_nước thuộc độ Tuyệt_mật và Tối_mật trình Thủ_tướng Chính_phủ quyết_định ; d . Quyết_định và giải mật_danh_mục bí_mật nhà_nước thuộc độ Mật sau khi thống_nhất với người đứng đầu hoặc người được ủy quyền của cơ_quan , tổ_chức có liên_quan ; đ . Thanh_tra , kiểm_tra , xử_lý vi_phạm và giải_quyết khiếu_nại , tố_cáo trong lĩnh_vực bảo_vệ bí_mật nhà_nước ; e . Giúp Chính_phủ sơ_kết , tổng_kết công_tác bảo_vệ bí_mật nhà_nước . Điều 12Trong phạm_vi nhiệm_vụ , quyền_hạn của mình , người đứng đầu hoặc người được ủy quyền của cơ_quan , tổ_chức và Chủ_tịch ủy_ban nhân_dân các cấp có trách_nhiệm sau đây : 1 . Tổ_chức thực_hiện công_tác bảo_vệ bí_mật nhà_nước theo quy_định của Pháp_lệnh này và các văn_bản pháp_luật khác có liên_quan ; 2 . Ban_hành và tổ_chức thực_hiện nội_quy bảo_vệ bí_mật nhà_nước theo quy_định của Chính_phủ ; 3 . Lập danh_mục , thay_đổi độ mật , giải mật bí_mật nhà_nước gửi cấp có thẩm_quyền quyết_định ; 4 . Bố_trí cán_bộ làm công_tác bảo_vệ bí_mật nhà_nước theo quy_định của Chính_phủ ; 5 . Tuyên_truyền , giáo_dục những người thuộc quyền quản_lý của mình nâng cao trách_nhiệm , cảnh_giác và nghiêm_chỉnh chấp_hành pháp_luật về bảo_vệ bí_mật nhà_nước ; 6 . Thực_hiện chế_độ báo_cáo về công_tác bảo_vệ bí_mật nhà_nước theo quy_định của Chính_phủ . Điều 13Bộ Quốc_phòng có trách_nhiệm tổ_chức thực_hiện công_tác bảo_vệ bí_mật nhà_nước trong các...
--------------------------------------------------------------------------------
Rank 2:
File Path: /content/vbpl/full_141824.html
Cosine Similarity: 0.6101
Content Preview: c ) Tổ_chức bồi_dưỡng nghiệp_vụ , kiến_thức bảo_vệ bí_mật nhà_nước ;
 d ) Phòng , chống vi_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước ;
 đ ) Kiểm_tra , thanh_tra , giải_quyết khiếu_nại , tố_cáo và xử_lý vi_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước ;
 e ) Thực_hiện hợp_tác quốc_tế về bảo_vệ bí_mật nhà_nước theo phân_công của Chính_phủ ;
 g ) Quy_định mẫu dấu chỉ độ mật , văn_bản xác_định độ mật , hình_thức khác chỉ độ mật và mẫu giấy_tờ về bảo_vệ bí_mật nhà_nước .
 3 . Văn_phòng Trung_ương Đảng và ban đảng , đảng_đoàn , ban cán_sự đảng và đảng_ủy trực_thuộc trung_ương ; cơ_quan trung_ương của tổ_chức chính_trị - xã_hội , tổ_chức xã_hội ; Hội_đồng Dân_tộc , Ủy_ban của Quốc_hội , cơ_quan thuộc Ủy_ban Thường_vụ Quốc_hội , Văn_phòng Quốc_hội ; Văn_phòng Chủ_tịch nước ; Bộ , cơ_quan ngang Bộ , cơ_quan thuộc Chính_phủ ; Tòa_án nhân_dân tối_cao ; Viện kiểm_sát_nhân_dân tối_cao ; Kiểm_toán nhà_nước ; tỉnh_ủy , thành_ủy , Đoàn đại_biểu Quốc_hội , Hội_đồng nhân_dân , Ủy_ban nhân_dân tỉnh , thành_phố trực_thuộc Trung_ương , trong phạm_vi nhiệm_vụ , quyền_hạn của mình , có trách_nhiệm sau đây :
 a ) Tổ_chức thực_hiện công_tác bảo_vệ bí_mật nhà_nước ;
 b ) Chủ_trì xây_dựng và trình cơ_quan có thẩm_quyền ban_hành hoặc ban_hành theo thẩm_quyền văn_bản chuyên_ngành liên_quan đến bảo_vệ bí_mật nhà_nước thuộc phạm_vi quản_lý phù_hợp với quy_định của Luật này ;
 c ) Ban_hành và tổ_chức thực_hiện quy_chế bảo_vệ bí_mật nhà_nước của cơ_quan , tổ_chức , địa_phương ;
 d ) Kiểm_tra , thanh_tra , giải_quyết khiếu_nại , tố_cáo và xử_lý vi_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước đối_với cơ_quan , tổ_chức , cá_nhân trực_thuộc ;
 đ ) Phân_công người thực_hiện nhiệm_vụ bảo_vệ bí_mật nhà_nước , thực_hiện chế_độ báo_cáo về công_tác bảo_vệ bí_mật nhà_nước theo quy_định của Chính_phủ .
 4 . Bộ Quốc_phòng chịu trách_nhiệm trước Chính_phủ thực_hiện quản_lý nhà_nước về bảo_vệ bí_mật nhà_nước thuộc phạm_vi quản_lý và thực_hiện trách_nhiệm quy_định tại khoản 3 Điều này ....
--------------------------------------------------------------------------------
Rank 3:
File Path: /content/vbpl/full_5585.html
Cosine Similarity: 0.5415
Content Preview: nhà_nước trong các cơ_quan , đơn_vị thuộc phạm_vi quản_lý của mình theo quy_định của Chính_phủ . Điều 14Chính phủ quy_định việc bảo_vệ bí_mật nhà_nước trong hoạt_động xuất_bản , báo_chí và thông_tin đại_chúng khác phù_hợp với quy_định của Pháp_lệnh này . Điều 15Nội dung bí_mật nhà_nước nếu truyền đưa bằng phương_tiện viễn_thông và máy_tính thì phải được mã_hóa theo quy_định của pháp_luật về cơ_yếu . Điều 16Công trình khoa_học , phát_minh , sáng_chế , giải_pháp hữu_ích của cơ_quan , tổ_chức hoặc công_dân có liên_quan đến nội_dung bí_mật nhà_nước phải đăng_ký tại cơ_quan nhà_nước có thẩm_quyền và được bảo_vệ theo quy_định của pháp_luật . Điều 17Cơ quan , tổ_chức , công_dân Việt_Nam tiếp_xúc với tổ_chức , cá_nhân nước_ngoài phải tuân_thủ các quy_định của pháp_luật về bảo_vệ bí_mật nhà_nước ; khi tiến_hành chương_trình hợp_tác quốc_tế có liên_quan đến bí_mật nhà_nước thì phải được sự đồng_ý của cơ_quan nhà_nước có thẩm_quyền về bảo_vệ bí_mật nhà_nước . Điều 18Người làm công_tác bảo_vệ bí_mật nhà_nước phải có phẩm_chất tốt , có trình_độ chuyên_môn nghiệp_vụ , có năng_lực hoàn_thành nhiệm_vụ được giao và phải cam_kết bảo_vệ bí_mật nhà_nước . Người được giao nhiệm_vụ tiếp_xúc với bí_mật nhà_nước phải cam_kết bảo_vệ bí_mật nhà_nước . CHƯƠNG IVKHEN THƯỞNG VÀ XỬ_LÝ VI_PHẠM Điều 19Cơ quan , tổ_chức và công_dân có thành_tích bảo_vệ bí_mật nhà_nước thì được khen_thưởng theo quy_định của pháp_luật . Điều 20Người nào vi_phạm các quy_định của Pháp_lệnh này và các quy_định khác của pháp_luật về bảo_vệ bí_mật nhà_nước thì tùy theo tính_chất , mức_độ vi_phạm mà bị xử_lý kỷ_luật , xử_lý vi_phạm hành_chính hoặc truy_cứu trách_nhiệm hình_sự ; nếu gây thiệt_hại thì phải bồi_thường theo quy_định của pháp_luật . CHƯƠNG V ĐIỀU_KHOẢN THI HÀNHĐiều 21Pháp lệnh này có hiệu_lực kể từ ngày 01 tháng 4 năm 2001 . Pháp_lệnh này thay_thế Pháp_lệnh bảo_vệ bí_mật nhà_nước ngày 28 tháng 10 năm 1991 . Những quy_định trước đây trái với Pháp_lệnh này đều bãi_bỏ . Điều 22Chính phủ quy_định chi_tiết và...
--------------------------------------------------------------------------------
Rank 4:
File Path: /content/vbpl/full_26724.html
Cosine Similarity: 0.5596
Content Preview: 2 . Bộ Quốc_phòng chủ_trì và có trách_nhiệm chỉ_đạo các đơn_vị cơ_sở trực_thuộc Bộ tổ_chức thực_hiện tuyên_truyền , phổ_biến pháp_luật và kiến_thức phòng cháy và chữa_cháy đối_với cơ_sở quốc_phòng , huấn_luyện nghiệp_vụ cho cán_bộ , sĩ_quan , quân_nhân chuyên_nghiệp , hạ_sĩ_quan , chiến_sỹ và công_nhân_viên của Bộ Quốc_phòng ; phối_hợp với lực_lượng Cảnh_sát phòng cháy , chữa_cháy và cứu nạn , cứu_hộ để tổ_chức kiểm_tra , sát_hạch và cấp giấy chứng_nhận huấn_luyện nghiệp_vụ phòng cháy và chữa_cháy cho các đối_tượng huấn_luyện đủ điều_kiện theo quy_định .
 Điều 8 . Tổ_chức lực_lượng và xây_dựng phong_trào phòng cháy và chữa_cháy
 1 . Bộ Công_an có trách_nhiệm chỉ_đạo lực_lượng Cảnh_sát phòng cháy , chữa_cháy và cứu nạn , cứu_hộ phối_hợp tổ_chức hướng_dẫn nghiệp_vụ cho lực_lượng phòng cháy và chữa_cháy ở các cơ_sở quốc_phòng theo kế_hoạch đã được thống_nhất .
 2 . Bộ Quốc_phòng chủ_trì phối_hợp với Bộ Công_an chỉ_đạo các cơ_sở quốc_phòng tổ_chức xây_dựng lực_lượng phòng cháy và chữa_cháy ; xây_dựng phong_trào quần_chúng trong các cơ_sở quốc_phòng tham_gia hoạt_động phòng cháy và chữa_cháy ; tổ_chức huấn_luyện nghiệp_vụ và trang_bị phương_tiện phòng cháy và chữa_cháy cho lực_lượng phòng cháy và chữa_cháy của các cơ_sở quốc_phòng .
 Điều 9 . Thẩm duyệt về phòng cháy và chữa_cháy
 1 . Bộ Công_an chịu trách_nhiệm chỉ_đạo tổ_chức thực_hiện thẩm duyệt , kiểm_tra thi_công và nghiệm_thu về phòng cháy , chữa_cháy đối_với các cơ_sở quốc_phòng hoạt_động phục_vụ kinh_tế - xã_hội và phương_tiện giao_thông cơ_giới có yêu_cầu đặc_biệt về bảo_đảm an_toàn phòng cháy và chữa_cháy được chế_tạo hoặc hoán cải chuyên_dùng cho hoạt_động phục_vụ kinh_tế - xã_hội ....
--------------------------------------------------------------------------------
Rank 5:
File Path: /content/vbpl/full_32521.html
Cosine Similarity: 0.5740
Content Preview: 2 . Bộ Quốc_phòng chủ_trì , phối_hợp với Bộ Công_an , Bộ Giáo_dục và Đào_tạo giúp Chính_phủ thực_hiện quản_lý nhà_nước về giáo_dục quốc_phòng và an_ninh .
 3 . Bộ , cơ_quan ngang bộ phối_hợp với Bộ Quốc_phòng thực_hiện quản_lý nhà_nước về giáo_dục quốc_phòng và an_ninh theo thẩm_quyền .
 4 . Ủy_ban nhân_dân các cấp trong phạm_vi nhiệm_vụ , quyền_hạn của mình thực_hiện quản_lý nhà_nước về giáo_dục quốc_phòng và an_ninh tại địa_phương .
 Điều 34 . Nhiệm_vụ , quyền_hạn của Bộ Quốc_phòng
 1 . Ban_hành theo thẩm_quyền hoặc trình cấp có thẩm_quyền ban_hành_văn_bản quy_phạm_pháp_luật về giáo_dục quốc_phòng và an_ninh .
 2 . Chủ_trì , phối_hợp với Bộ Giáo_dục và Đào_tạo , bộ , cơ_quan ngang bộ có liên_quan lập quy_hoạch hệ_thống trung_tâm giáo_dục quốc_phòng và an_ninh trên phạm_vi cả nước , trình Thủ_tướng Chính_phủ xem_xét , quyết_định ; quyết_định thành_lập trung_tâm giáo_dục quốc_phòng và an_ninh thuộc nhà_trường quân_đội .
 3 . Chủ_trì , phối_hợp với Bộ Công_an , Bộ Giáo_dục và Đào_tạo , bộ , cơ_quan ngang bộ , cơ_quan , tổ_chức ở trung_ương có liên_quan :
 a ) Quy_định chương_trình khung giáo_dục quốc_phòng và an_ninh cho người học trong trường của cơ_quan nhà_nước , tổ_chức chính_trị , tổ_chức chính_trị - xã_hội ; chương_trình khung bồi_dưỡng kiến_thức quốc_phòng và an_ninh cho cá_nhân tiêu_biểu , người có uy_tín trong cộng_đồng dân_cư ; chương_trình , nội_dung bồi_dưỡng kiến_thức quốc_phòng và an_ninh cho đối_tượng trong cơ_quan , tổ_chức của Nhà_nước , tổ_chức chính_trị , tổ_chức chính_trị - xã_hội , doanh_nghiệp ngoài khu_vực nhà_nước , đơn_vị sự_nghiệp ngoài công_lập ;
 b ) Quy_định tiêu_chuẩn tuyên_truyền_viên giáo_dục quốc_phòng và an_ninh ;
 c ) Biên_soạn giáo_trình , tài_liệu phổ_biến kiến_thức quốc_phòng và an_ninh phù_hợp với tình_hình , nhiệm_vụ xây_dựng và bảo_vệ Tổ_quốc trong từng thời_kỳ ....
--------------------------------------------------------------------------------
'''

'''
Query 2: "Bí mật Nhà nước là"
Rank 1:
File Path: /content/vbpl/full_5585.html
Cosine Similarity: 0.5551
Content Preview: 5Bí mật nhà_nước trong phạm_vi sau đây thuộc độ Tuyệt_mật : 1 . Chiến_lược an_ninh quốc_gia ; kế_hoạch phòng_thủ đất_nước ; kế_hoạch động_viên đối_phó với chiến_tranh ; các loại vũ_khí , phương_tiện có ý_nghĩa quyết_định khả_năng phòng_thủ đất_nước ; 2 . Các chủ_trương , chính_sách về đối_nội , đối_ngoại của Đảng Cộng_sản Việt_Nam và Nhà_nước Cộng_hòa xã_hội chủ_nghĩa Việt_Nam không công_bố hoặc chưa công_bố . Những tin của nước_ngoài hoặc của các tổ_chức quốc_tế chuyển_giao cho Việt_Nam mà Chính_phủ xác_định thuộc độ Tuyệt_mật ; 3 . Tổ_chức và hoạt_động tình_báo , phản_gián do Chính_phủ quy_định ; 4 . Mật_mã quốc_gia ; 5 . Dự_trữ chiến_lược quốc_gia ; các số_liệu dự_toán , quyết_toán ngân_sách nhà_nước về những lĩnh_vực chưa công_bố ; kế_hoạch phát_hành tiền , khóa an_toàn của từng mẫu tiền và các loại giấy_tờ có giá_trị như tiền ; phương_án , kế_hoạch thu đổi tiền chưa công_bố ; 6 . Khu_vực , địa_điểm cấm ; tin , tài_liệu khác mà Chính_phủ xác_định thuộc độ Tuyệt_mật . Điều 6Bí mật nhà_nước trong phạm_vi sau đây thuộc độ Tối_mật : 1 . Các cuộc đàm_phán và tiếp_xúc cấp cao giữa nước ta với nước_ngoài hoặc các tổ_chức quốc_tế về chính_trị , quốc_phòng , an_ninh , đối_ngoại , kinh_tế , khoa_học , công_nghệ và các lĩnh_vực khác chưa công_bố . Những tin của nước_ngoài hoặc của các tổ_chức quốc_tế chuyển_giao cho Việt_Nam mà Chính_phủ xác_định thuộc độ Tối_mật ; 2 . Tổ_chức hoạt_động , trang_bị , phương_án tác_chiến của các đơn_vị vũ_trang nhân_dân , trừ tổ_chức và hoạt_động được quy_định tại khoản 3 Điều 5 của Pháp_lệnh này ; phương_án sản_xuất , vận_chuyển và cất_giữ vũ_khí ; công_trình quan_trọng phòng_thủ biên_giới , vùng_trời , vùng_biển , hải_đảo ; 3 . Bản_đồ quân_sự ; tọa_độ điểm hạng I , hạng II nhà_nước của mạng_lưới quốc_gia hoàn_chỉnh cùng với các ghi_chú điểm kèm theo . Vị_trí và trị_số độ cao các mốc chính của các trạm khí_tượng , thủy_văn , hải_văn ; số_liệu độ cao và số không tuyệt_đối của các mốc hải_văn ; 4 . Số_lượng tiền in , phát_hành ; tiền dự_trữ...
--------------------------------------------------------------------------------
Rank 2:
File Path: /content/vbpl/full_141824.html
Cosine Similarity: 0.5074
Content Preview: 2011 đến 2020

 CSDL quốc_gia về VBPL »
 CSDL Trung_ương »
 Văn_bản pháp_luật »
 Luật 29 / 2018 / QH14
 Toàn_văn
 Thuộc_tính
 Lịch_sử
 VB liên_quan
 Lược_đồ
 Tải_về
 Bản_in
 Hiệu_lực :
 Còn hiệu_lực
 Ngày có hiệu_lực :
 01 / 07 / 2020
 QUỐC_HỘI
 Số :
 29 / 2018 / QH14
 CỘNG_HOÀ XÃ_HỘI CHỦ_NGHĨA VIỆT_NAM
 Độc_lập - Tự_do - Hạnh_phúc
 Hà_Nội ,
 ngày
 15 tháng
 11 năm
 2018
 LUẬT
 BẢO_VỆ BÍ_MẬT NHÀ_NƯỚC
 Căn_cứ Hiến_pháp nước Cộng_hòa xã_hội chủ_nghĩa Việt_Nam ;
 Quốc_hội ban_hành Luật Bảo_vệ bí_mật nhà_nước .
 Chương_I
 NHỮNG QUY_ĐỊNH CHUNG
 Điều 1 . Phạm_vi điều_chỉnh
 Luật này quy_định về bí_mật nhà_nước , hoạt_động bảo_vệ bí_mật nhà_nước và trách_nhiệm của cơ_quan , tổ_chức , cá_nhân có liên_quan .
 Điều 2 . Giải_thích từ_ngữ
 Trong Luật này , các từ_ngữ dưới đây được hiểu như sau :
 1 . Bí_mật nhà_nước là thông_tin có nội_dung quan_trọng do người đứng đầu_cơ_quan , tổ_chức có thẩm_quyền xác_định căn_cứ vào quy_định của Luật này , chưa công_khai , nếu bị lộ , bị mất có_thể gây nguy_hại đến lợi_ích quốc_gia , dân_tộc .
 Hình_thức chứa bí_mật nhà_nước bao_gồm tài_liệu , vật , địa_điểm , lời_nói , hoạt_động hoặc các dạng khác .
 2 . Bảo_vệ bí_mật nhà_nước là việc cơ_quan , tổ_chức , cá_nhân sử_dụng lực_lượng , phương_tiện , biện_pháp để phòng , chống xâm_phạm bí_mật nhà_nước .
 3 . Lộ bí_mật nhà_nước là trường_hợp người không có trách_nhiệm biết được bí_mật nhà_nước .
 4 . Mất bí_mật nhà_nước là trường_hợp tài_liệu , vật chứa bí_mật nhà_nước không còn thuộc sự quản_lý của cơ_quan , tổ_chức , cá_nhân có trách_nhiệm quản_lý .
 Điều 3 . Nguyên_tắc bảo_vệ bí_mật nhà_nước
 1 . Đặt dưới sự lãnh_đạo của Đảng Cộng_sản Việt_Nam , sự quản_lý thống_nhất của Nhà_nước ; phục_vụ nhiệm_vụ xây_dựng và bảo_vệ Tổ_quốc , phát_triển kinh_tế - xã_hội , hội_nhập quốc_tế của đất_nước ; bảo_vệ lợi_ích quốc_gia , dân_tộc , quyền và lợi_ích hợp_pháp của cơ_quan , tổ_chức , cá_nhân ....
--------------------------------------------------------------------------------
Rank 3:
File Path: /content/vbpl/full_96122.html
Cosine Similarity: 0.4842
Content Preview: Điều 338 . Tội vô_ý làm lộ bí_mật nhà_nước ; tội làm mất vật , tài_liệu bí_mật nhà_nước
 1 . Người nào vô_ý làm lộ bí_mật nhà_nước hoặc làm mất vật , tài_liệu bí_mật nhà_nước , thì bị phạt cải_tạo không giam_giữ đến 03 năm hoặc phạt tù từ 06 tháng đến 03 năm .
 2 . Phạm_tội trong những trường_hợp sau đây , thì bị phạt tù từ 02 năm đến 07 năm :
 a ) Bí_mật nhà_nước thuộc độ tối_mật , tuyệt_mật ;
 b ) Gây tổn_hại về quốc_phòng , an_ninh , đối_ngoại , kinh_tế , văn_hóa chế_độ chính_trị , độc_lập , chủ_quyền , thống_nhất và toàn_vẹn lãnh_thổ .
 3 . Người phạm_tội còn có_thể bị cấm đảm_nhiệm chức_vụ , cấm hành_nghề hoặc làm công_việc nhất_định từ 01 năm đến 05 năm .
 Điều 339 . Tội giả_mạo chức_vụ , cấp_bậc , vị_trí công_tác
 Người nào giả_mạo chức_vụ , cấp_bậc , vị_trí công_tác thực_hiện_hành_vi trái pháp_luật nhưng không nhằm mục_đích chiếm_đoạt tài_sản , thì bị phạt cải_tạo không giam_giữ đến 02 năm hoặc phạt tù từ 03 tháng đến 02 năm .
 Điều 340 . Tội sửa_chữa và sử_dụng giấy chứng_nhận , các tài_liệu của cơ_quan , tổ_chức
 1 . Người nào sửa_chữa , làm sai_lệch nội_dung hộ_chiếu , thị_thực , hộ_khẩu , hộ_tịch , các loại giấy chứng_nhận hoặc tài_liệu của cơ_quan , tổ_chức và sử_dụng giấy_tờ đó thực_hiện tội_phạm hoặc đã bị xử_phạt vi_phạm hành_chính về hành_vi này hoặc đã bị kết_án về tội này , chưa được xóa_án tích mà còn vi_phạm , thì bị phạt cảnh_cáo , phạt tiền từ 10.000.000 đồng đến 50.000.000 đồng , phạt cải_tạo không giam_giữ đến 03 năm hoặc phạt tù từ 06 tháng đến 03 năm .
 2 . Phạm_tội thuộc một trong các trường_hợp sau đây , thì bị phạt tù từ 02 năm đến 05 năm :
 a ) Phạm_tội 02 lần trở lên ;
 b ) Sử_dụng giấy_tờ , tài_liệu đã bị sửa_chữa hoặc làm sai_lệch thực_hiện tội_phạm rất nghiêm_trọng hoặc tội đặc_biệt nghiêm_trọng .
 3 . Người phạm_tội còn có_thể bị phạt tiền từ 5.000.000 đồng đến 20.000.000 đồng , cấm đảm_nhiệm chức_vụ , cấm hành_nghề hoặc làm công_việc nhất_định từ 01 năm đến 05 năm ....
--------------------------------------------------------------------------------
Rank 4:
File Path: /content/vbpl/full_141824.html
Cosine Similarity: 0.4893
Content Preview: 2 . Bảo_vệ bí_mật nhà_nước là trách_nhiệm của mọi cơ_quan , tổ_chức , cá_nhân .
 3 . Việc quản_lý , sử_dụng bí_mật nhà_nước bảo_đảm đúng mục_đích , thẩm_quyền , trình_tự , thủ_tục theo quy_định của pháp_luật .
 4 . Chủ_động phòng_ngừa ; kịp_thời phát_hiện , ngăn_chặn , xử_lý nghiêm hành_vi vi_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước .
 5 . Bí_mật nhà_nước được bảo_vệ theo thời_hạn quy_định của Luật này , bảo_đảm quyền tiếp_cận thông_tin của công_dân theo quy_định của pháp_luật .
 Điều 4 . Hợp_tác quốc_tế về bảo_vệ bí_mật nhà_nước
 1 . Hợp_tác quốc_tế về bảo_vệ bí_mật nhà_nước được thực_hiện theo nguyên_tắc tôn_trọng độc_lập , chủ_quyền và toàn_vẹn lãnh_thổ , không can_thiệp vào công_việc nội_bộ của nhau , bình_đẳng và cùng có lợi .
 2 . Nội_dung hợp_tác quốc_tế về bảo_vệ bí_mật nhà_nước bao_gồm :
 a ) Ký_kết và tổ_chức thực_hiện điều_ước quốc_tế , thỏa_thuận quốc_tế về bảo_vệ bí_mật nhà_nước ;
 b ) Thực_hiện chương_trình , dự_án hợp_tác quốc_tế về bảo_vệ bí_mật nhà_nước ;
 c ) Đào_tạo , nghiên_cứu , ứng_dụng khoa_học và công_nghệ , kỹ_thuật phục_vụ công_tác bảo_vệ bí_mật nhà_nước ;
 d ) Phòng , chống vi_phạm_pháp_luật về bảo_vệ bí_mật nhà_nước ;
 đ ) Hoạt_động hợp_tác quốc_tế khác về bảo_vệ bí_mật nhà_nước .
 Điều 5 . Các hành_vi bị nghiêm_cấm trong bảo_vệ bí_mật nhà_nước
 1 . Làm lộ , chiếm_đoạt , mua , bán bí_mật nhà_nước ; làm sai_lệch , hư_hỏng , mất tài_liệu , vật chứa bí_mật nhà_nước .
 2 . Thu_thập , trao_đổi , cung_cấp , chuyển_giao bí_mật nhà_nước trái pháp_luật ; sao , chụp , lưu_giữ , vận_chuyển , giao , nhận , thu_hồi , tiêu_hủy tài_liệu , vật chứa bí_mật nhà_nước trái pháp_luật .
 3 . Mang tài_liệu , vật chứa bí_mật nhà_nước ra khỏi nơi lưu_giữ trái pháp_luật .
 4 . Lợi_dụng , lạm_dụng việc bảo_vệ bí_mật nhà_nước , sử_dụng bí_mật nhà_nước để thực_hiện , che_giấu hành_vi vi_phạm_pháp_luật , xâm_phạm quyền và lợi_ích hợp_pháp hoặc cản_trở hoạt_động của cơ_quan , tổ_chức , cá_nhân ....
--------------------------------------------------------------------------------
Rank 5:
File Path: /content/vbpl/full_141824.html
Cosine Similarity: 0.4496
Content Preview: 5 . Soạn_thảo , lưu_giữ tài_liệu có chứa nội_dung bí_mật nhà_nước trên máy_tính hoặc thiết_bị khác đã kết_nối hoặc đang kết_nối với mạng Internet , mạng máy_tính , mạng viễn_thông , trừ trường_hợp_lưu_giữ bí_mật nhà_nước theo quy_định của pháp_luật về cơ_yếu .
 6 . Truyền đưa bí_mật nhà_nước trên phương_tiện thông_tin , viễn_thông trái với quy_định của pháp_luật về cơ_yếu .
 7 . Chuyển mục_đích sử_dụng máy_tính , thiết_bị khác đã dùng để soạn_thảo , lưu_giữ , trao_đổi bí_mật nhà_nước khi chưa loại_bỏ bí_mật nhà_nước .
 8 . Sử_dụng thiết_bị có tính_năng thu , phát tín_hiệu , ghi_âm , ghi_hình trong hội_nghị , hội_thảo , cuộc họp có nội_dung bí_mật nhà_nước dưới mọi hình_thức khi chưa được người có thẩm_quyền cho phép .
 9 . Đăng_tải , phát_tán bí_mật nhà_nước trên phương_tiện thông_tin đại_chúng , mạng Internet , mạng máy_tính và mạng viễn_thông .
 Điều 6 . Kinh_phí , cơ_sở vật_chất phục_vụ bảo_vệ bí_mật nhà_nước
 1 . Kinh_phí , cơ_sở vật_chất phục_vụ bảo_vệ bí_mật nhà_nước do ngân_sách nhà_nước bảo_đảm .
 2 . Việc quản_lý , sử_dụng kinh_phí , cơ_sở vật_chất phục_vụ bảo_vệ bí_mật nhà_nước thực_hiện theo quy_định của pháp_luật .
 Chương II
 PHẠM_VI , PHÂN_LOẠI ,
 BAN_HÀNH DANH_MỤC BÍ_MẬT NHÀ_NƯỚC
 Điều 7 . Phạm_vi bí_mật nhà_nước
 Phạm_vi bí_mật nhà_nước là giới_hạn thông_tin quan_trọng trong các lĩnh_vực sau đây chưa công_khai , nếu bị lộ , bị mất có_thể gây nguy_hại đến lợi_ích quốc_gia , dân_tộc :
 1 . Thông_tin về chính_trị :
 a ) Chủ_trương , chính_sách của Đảng và Nhà_nước về đối_nội , đối_ngoại ;
 b ) Hoạt_động của Ban_Chấp_hành Trung_ương , Bộ Chính_trị , Ban Bí_thư và lãnh_đạo Đảng , Nhà_nước ;
 c ) Chiến_lược , đề_án về dân_tộc , tôn_giáo và công_tác dân_tộc , tôn_giáo liên_quan đến bảo_vệ an_ninh quốc_gia , bảo_đảm trật_tự , an_toàn xã_hội ;
 d ) Thông_tin có tác_động tiêu_cực đến tình_hình chính_trị , kinh_tế - xã_hội .
 2 . Thông_tin về quốc_phòng , an_ninh , cơ_yếu :...
--------------------------------------------------------------------------------
'''